In [130]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import os
import random as rnd
import statistics as stat
import numpy as np

# Model

Will ich das ganze hier Objektorientiert angehen? Ich könnte jeden Akteur als Objekt betrachten, mit assesment und peers. Ich frage mich aber auch schon, ob das Programm dann schnell läuft, oder ob es sich noch irgendwie optimieren lässt.

Große Frage: Wie programmiere ich die Zeitschritte optimal?

Ich will:

- update the assesment of every agent to their observation and the mean of their peers
- update the peers of every agent

Dabei frage ich mich, ob ich die peers von jedem agent schneller finden kann als, indem ich jedes mal jeden anderen Akteur betrachte. (Das ist wichtig, weil die Simulationen sonst lange dauern)

Das ginge, wenn ich die Akteure in einer Liste speichere, die nach ihrem assesment sortiert ist. Dann muss ich mich nur vom gegenwärtigen Akteur nach links und rechts vortasten und kann aufhören, sobald ich zwei finde, die weiter weg sind als epsilon. Ich müsste also die Liste sortieren, nachdem ich die assesments geupdated habe. Könnte sein, dass ich dabei ein paar Schritte spare.

Ich programmiere es jetzt erstmal brute force und wenn ich Geschwindigkeitsprobleme kriege, probier ich die Version.

In [131]:
# --- globals --- #

# let there be n agents with random initial assesements
# every agent is represented by an integer in [0, n)
n = 10
epsilon = .3
alpha = .4
noise = .5
tau = .1
max_time = 5
data = []

In [132]:
# --- classes --- #

class Agent:
    def __init__(self, id):
        self.assesment = 1 - rnd.random() # values in (0, 1]
        self.peers = set()
        self.id = id
    
    def __str__(self):
        return f"assesment: {self.assesment} and peers: {self.peers}"

    def __repr__(self):
        return f"(Agent [{self.id}]: {self.assesment:.2f}, {self.peers})"

    def update_peers(self, agent_list):
        for potential_peer in agent_list:
            if (potential_peer.id != self.id and 
                    abs(potential_peer.assesment - self.assesment < epsilon)):
                self.peers.add(potential_peer)

In [133]:
# --- functions --- #

def print_agent_list(agent_list):
    print("[")
    for agent in agent_list:
        representation = f"    (Agent [{agent.id}]: {agent.assesment:.2f}, " + "{"
        representation += ", ".join(f"[{peer.id}]" for peer in agent.peers) + "})"
    
        print(representation)
    print("]")

In [134]:
# --- main code for abm --- #

agents = []
for i in range(n):
    agent = Agent(i)
    agents.append(agent)
    data.append([agent.id, 0, agent.assesment])

print_agent_list(agents)
print(data)

[
    (Agent [0]: 0.97, {})
    (Agent [1]: 0.46, {})
    (Agent [2]: 0.01, {})
    (Agent [3]: 0.17, {})
    (Agent [4]: 0.76, {})
    (Agent [5]: 0.43, {})
    (Agent [6]: 0.48, {})
    (Agent [7]: 0.77, {})
    (Agent [8]: 0.32, {})
    (Agent [9]: 0.26, {})
]
[[0, 0, 0.9708462904552279], [1, 0, 0.459206252848514], [2, 0, 0.014178779597557867], [3, 0, 0.17452379686063435], [4, 0, 0.7597308613125231], [5, 0, 0.4320965960614652], [6, 0, 0.48089616833258886], [7, 0, 0.7732483671835412], [8, 0, 0.3221599983974409], [9, 0, 0.259132219923797]]


In [102]:
# for each agent let there be a set of agents with similar assesments
for i in range(n):
    agents[i].update_peers(agents)

print_agent_list(agents)

[
    (Agent [0]: 0.48, {[6], [1], [8], [2], [5], [3], [4], [9]})
    (Agent [1]: 0.08, {[6], [8], [5], [3], [4], [9]})
    (Agent [2]: 0.41, {[6], [1], [0], [8], [5], [3], [4], [9]})
    (Agent [3]: 0.38, {[6], [1], [0], [8], [2], [5], [4], [9]})
    (Agent [4]: 0.32, {[6], [1], [0], [8], [2], [5], [3], [9]})
    (Agent [5]: 0.27, {[6], [1], [0], [8], [2], [3], [4], [9]})
    (Agent [6]: 0.04, {[1], [8], [5], [4], [9]})
    (Agent [7]: 0.89, {[6], [1], [0], [8], [2], [5], [3], [4], [9]})
    (Agent [8]: 0.33, {[6], [1], [0], [2], [5], [3], [4], [9]})
    (Agent [9]: 0.22, {[6], [1], [0], [8], [2], [5], [3], [4]})
]


In [103]:
# for each time step
for u in range(1, max_time): # we did the fist step by setting everything up
    # update the assesment of each agent to a ratio between the agents observations and the mean of their peers
    for agent in agents:
        # observing
        observation = np.random.normal(tau, noise/2) # random value from bell curve with tau as 2 std deviations
        while observation <= 0 or observation > 1: # cutting off values outside (0, 1]
            observation = np.random.normal(tau, noise/2)
        
        if agent.peers:
            observation *= (1- alpha)

            # listening to peers
            collective_peer_assesment = alpha * stat.mean([peer.assesment for peer in agent.peers])

            agent.assesment = observation + collective_peer_assesment
        else:
            # TODO: unneccessary: I should be my own peer; also: copy the assesments and work on a dict that doesnt change while you do it
            agent.assesment = observation
    
        data.append([agent.id, u, agent.assesment])

    # update each agents peers
    for agent in agents:
        agent.update_peers(agents)
    
    print_agent_list(agents)

        

[
    (Agent [0]: 0.14, {[6], [1], [8], [2], [5], [7], [3], [4], [9]})
    (Agent [1]: 0.17, {[6], [0], [8], [2], [5], [7], [3], [4], [9]})
    (Agent [2]: 0.30, {[6], [1], [0], [8], [5], [7], [3], [4], [9]})
    (Agent [3]: 0.09, {[6], [1], [0], [8], [2], [5], [7], [4], [9]})
    (Agent [4]: 0.21, {[6], [1], [0], [8], [2], [5], [7], [3], [9]})
    (Agent [5]: 0.16, {[6], [1], [0], [8], [2], [7], [3], [4], [9]})
    (Agent [6]: 0.16, {[1], [0], [8], [2], [5], [7], [3], [4], [9]})
    (Agent [7]: 0.17, {[6], [1], [0], [8], [2], [5], [3], [4], [9]})
    (Agent [8]: 0.12, {[6], [1], [0], [2], [5], [7], [3], [4], [9]})
    (Agent [9]: 0.10, {[6], [1], [0], [8], [2], [5], [7], [3], [4]})
]
[
    (Agent [0]: 0.34, {[6], [1], [8], [2], [5], [7], [3], [4], [9]})
    (Agent [1]: 0.22, {[6], [0], [8], [2], [5], [7], [3], [4], [9]})
    (Agent [2]: 0.18, {[6], [1], [0], [8], [5], [7], [3], [4], [9]})
    (Agent [3]: 0.31, {[6], [1], [0], [8], [2], [5], [7], [4], [9]})
    (Agent [4]: 0.13, {[6], 

In [ ]:
# --- figuring out how to store the data --- #

# seaborn
dots = sns.load_dataset("dots")
sns.relplot(
    data=results, kind="line",
    x="time", y="assesment"
    hue="agent",
    facet_kws=dict(sharex=False),#wat is dis?
)

# ich will...
# Alle Daten in pd dataframe
# ein column: time
# ein column: agent
# ein column: assesment
# dann einen Graphen mit mehreren Linien, die auf der x-Achse die Zeit und auf y das assesment haben

In [114]:
# "agent", "time", "assesment"

dict = {
    1: [1, 2, 3],
    2: [2, 4, 6],
    3: [3, 9, 18],
    4: [4, 8, 16]
}

arr = [
    [1, 2, 3],
    [10, 20, 30],
    [100, 200, 300]
]

arr.append([1000, 2000, 3000])

pd.DataFrame(arr, columns=["one", "two", "three"])

,one,two,three
0,1,2,3
1,10,20,30
2,100,200,300
3,1000,2000,3000


In [137]:
for i in range(1, 10):
    print(i)

1
2
3
4
5
6
7
8
9
